In [14]:
from google.colab import drive
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy import stats
import numpy as np
import logging
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from datetime import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime
import matplotlib.dates as mdates
import os

In [15]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Set the base path to the desired directory on Google Drive
base_path = '/content/drive/MyDrive/Study_1_Data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
def read_csv(file_path):
    data = pd.read_csv(file_path)
    return data

In [17]:
def process_data(data, columns_to_remove):

    processed_data = data.drop(columns=columns_to_remove).values
    return processed_data

In [18]:
def construct_3d_array(base_dir, participants, simulations, columns_to_remove_hr, columns_to_remove_gsr, columns_to_remove_head, columns_to_remove_eye):
    """
    Construct 3D array from CSV files.
    """
    num_rows = 180  # Define number of rows to keep (last 180 rows)
    arrays_3d = []

    for participant in participants:
        participant_id = f"{int(participant):02d}"  # Format participant number to two digits

        valid_simulations = []

        for simulation in simulations:
            hr_file_path = os.path.join(base_dir, participant_id, simulation, f'HR{simulation.capitalize()}.csv')
            gsr_file_path = os.path.join(base_dir, participant_id, simulation, f'EDA{simulation.capitalize()}_downsampled.csv')
            head_file_path = os.path.join(base_dir, participant_id, simulation, 'head_tracking_downsampled.csv')
            eye_file_path = os.path.join(base_dir, participant_id, simulation, 'eye_tracking_downsampled.csv')

            # Check if all files exist
            if all(os.path.exists(file) for file in [hr_file_path, gsr_file_path, head_file_path, eye_file_path]):
                valid_simulations.append(simulation)

        num_valid_simulations = len(valid_simulations)
        if num_valid_simulations == 0:
            continue  # Skip this participant if no valid simulations are found

        array_3d = np.zeros((num_valid_simulations, num_rows, 47)) # hr=1, gsr=1, head=15-3, eye=41-8 total columns after removing columns= 48

        for s_idx, simulation in enumerate(valid_simulations):
            # Process hr data
            hr_file_path = os.path.join(base_dir, participant_id, simulation, f'HR{simulation.capitalize()}.csv')
            hr_data = read_csv(hr_file_path)
            processed_hr_data = process_data(hr_data, columns_to_remove_hr)
            processed_hr_data = processed_hr_data[-num_rows:]  # Keep only the last 180 rows

            # Process gsr data
            gsr_file_path = os.path.join(base_dir, participant_id, simulation, f'EDA{simulation.capitalize()}_downsampled.csv')
            gsr_data = read_csv(gsr_file_path)
            processed_gsr_data = process_data(gsr_data, columns_to_remove_gsr)
            processed_gsr_data = processed_gsr_data[-num_rows:]  # Keep only the last 180 rows

            # Process head data
            head_file_path = os.path.join(base_dir, participant_id, simulation, 'head_tracking_downsampled.csv')
            head_data = read_csv(head_file_path)
            processed_head_data = process_data(head_data, columns_to_remove_head)
            processed_head_data = processed_head_data[-num_rows:]  # Keep only the last 180 rows

            # Process eye data
            eye_file_path = os.path.join(base_dir, participant_id, simulation, 'eye_tracking_downsampled.csv')
            eye_data = read_csv(eye_file_path)
            processed_eye_data = process_data(eye_data, columns_to_remove_eye)
            processed_eye_data = processed_eye_data[-num_rows:]  # Keep only the last 180 rows

            # Combine processed data
            combined_data = np.concatenate((processed_hr_data, processed_gsr_data, processed_head_data, processed_eye_data), axis=1)

            array_3d[s_idx, :, :] = combined_data

        arrays_3d.append(array_3d)

    return arrays_3d


In [19]:
sample_size=60
simulations = ['flat','noise','bumps']
participants = [str(i) for i in range(1, 27)]  # Participants 101 to 127
columns_to_remove_hr = []
columns_to_remove_gsr = []
columns_to_remove_eye = ['#Frame','Time', 'Unnamed: 40','ConvergenceValid','Left_Eye_Closed','Right_Eye_Closed','LocalGazeValid','WorldGazeValid']
columns_to_remove_head = ['#Frame','Time', 'Unnamed: 14']

In [20]:
def calculate_total_ssq(csv_file):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)
    n_columns = [0, 5, 6, 7, 8, 14, 15]
    o_columns = [0, 1, 2, 3, 4, 8, 10]
    d_columns = [4, 7, 9, 10, 11, 12, 13]

    # Calculate sum for each specified set of columns
    n_val = df.iloc[0, n_columns].sum()
    o_val = df.iloc[0, o_columns].sum()
    d_val = df.iloc[0, d_columns].sum()

    return n_val, o_val, d_val

In [21]:
def merge_ssq_column(conditions,participants):
  directories = []
  total_ssq_values = []
  for participant in participants:
      participant = f"{int(participant):02d}"
      for condition in conditions:
          directory = os.path.join(base_path, participant, condition)
          directories.append(directory)

  # Loop through each directory
  for directory in directories:
      # Check if the directory exists
      if not os.path.exists(directory):
          continue

      # Get all CSV files in the directory that are named 'ssq.csv'
      csv_files = [file for file in os.listdir(directory) if file == 'ssq.csv']

      # Loop through each CSV file
      for csv_file in csv_files:
          csv_path = os.path.join(directory, csv_file)
          df = pd.read_csv(csv_path)
          n_val,o_val,d_val = calculate_total_ssq(csv_path)
          total_ssq_values.append([n_val, o_val, d_val])
          #ssq_values_participant = df.iloc[:, 0:17].values.flatten()   # Assuming SSQ values are in columns 1 to 16
          #total_ssq_values.append(ssq_values_participant)
  ssq_array = np.array(total_ssq_values)
  return ssq_array

def merge_total_ssq(conditions,participants):
  directories = []
  total_ssq_values = []
  for participant in participants:
      participant = f"{int(participant):02d}"
      for condition in conditions:
          directory = os.path.join(base_path, participant, condition)
          directories.append(directory)

  # Loop through each directory
  for directory in directories:
      # Check if the directory exists
      if not os.path.exists(directory):
          continue

      # Get all CSV files in the directory that are named 'ssq.csv'
      csv_files = [file for file in os.listdir(directory) if file == 'ssq.csv']

      # Loop through each CSV file
      for csv_file in csv_files:
          csv_path = os.path.join(directory, csv_file)
          n_val,o_val,d_val = calculate_total_ssq(csv_path)
          total_ssq = (n_val+o_val+d_val) * 3.74
          df = pd.read_csv(csv_path)
          df["total-ssq"] = total_ssq
          #print("csv_path: ",csv_path,"   ",total_ssq)
          total_ssq_values.append(total_ssq)
  # Create a DataFrame from the list of total SSQ values
  df_total_ssq = pd.DataFrame(total_ssq_values, columns=["total-ssq"])
  # Convert the list of total SSQ values to a NumPy array
  total_ssq_array = np.array(total_ssq_values)
  return total_ssq_array



In [22]:
participants_group_1 = [1,3,4,11,25]
participants_group_2 = [2,7,8,9,17]
participants_group_3 = [10,12,13,22,23]
participants_group_4 = [5,14,18,20,21]
participants_group_5 = [6,15,16,19,24,26]

arrays_group_1 = construct_3d_array(base_path, participants_group_1, simulations, columns_to_remove_hr, columns_to_remove_gsr, columns_to_remove_head, columns_to_remove_eye)
arrays_group_2 = construct_3d_array(base_path, participants_group_2, simulations, columns_to_remove_hr, columns_to_remove_gsr, columns_to_remove_head, columns_to_remove_eye)
arrays_group_3 = construct_3d_array(base_path, participants_group_3, simulations, columns_to_remove_hr, columns_to_remove_gsr, columns_to_remove_head, columns_to_remove_eye)
arrays_group_4 = construct_3d_array(base_path, participants_group_4, simulations, columns_to_remove_hr, columns_to_remove_gsr, columns_to_remove_head, columns_to_remove_eye)
arrays_group_5 = construct_3d_array(base_path, participants_group_5, simulations, columns_to_remove_hr, columns_to_remove_gsr, columns_to_remove_head, columns_to_remove_eye)

In [23]:
# Concatenate arrays along the first axis
input_group_1 = np.concatenate(arrays_group_1, axis=0)
input_group_2 = np.concatenate(arrays_group_2, axis=0)
input_group_3 = np.concatenate(arrays_group_3, axis=0)
input_group_4 = np.concatenate(arrays_group_4, axis=0)
input_group_5 = np.concatenate(arrays_group_5, axis=0)


In [24]:
output_group_1=merge_ssq_column(simulations,participants_group_1)
output_group_2=merge_ssq_column(simulations,participants_group_2)
output_group_3=merge_ssq_column(simulations,participants_group_3)
output_group_4=merge_ssq_column(simulations,participants_group_4)
output_group_5=merge_ssq_column(simulations,participants_group_5)

output_group_1 = np.squeeze(output_group_1)
output_group_2 = np.squeeze(output_group_2)
output_group_3 = np.squeeze(output_group_3)
output_group_4 = np.squeeze(output_group_4)
output_group_5 = np.squeeze(output_group_5)


output_total_ssq_group_1=merge_total_ssq(simulations,participants_group_1)
output_total_ssq_group_2=merge_total_ssq(simulations,participants_group_2)
output_total_ssq_group_3=merge_total_ssq(simulations,participants_group_3)
output_total_ssq_group_4=merge_total_ssq(simulations,participants_group_4)
output_total_ssq_group_5=merge_total_ssq(simulations,participants_group_5)

output_total_ssq_group_1=output_total_ssq_group_1.reshape(-1, 1)
output_total_ssq_group_2=output_total_ssq_group_2.reshape(-1, 1)
output_total_ssq_group_3=output_total_ssq_group_3.reshape(-1, 1)
output_total_ssq_group_4=output_total_ssq_group_4.reshape(-1, 1)
output_total_ssq_group_5=output_total_ssq_group_5.reshape(-1, 1)



In [25]:
def scale_input_data(input_train, input_test):
    # Get the shape of the input data
    num_samples_train, time_steps_train, num_features = input_train.shape
    num_samples_test, time_steps_test, _ = input_test.shape

    # Reshape the input data into 2D arrays
    flattened_train_data = input_train.reshape(-1, num_features)
    flattened_test_data = input_test.reshape(-1, num_features)

    # Initialize a MinMaxScaler object
    scaler = MinMaxScaler()

    # Fit the scaler on the training data and transform both train and test data
    scaled_train_data = scaler.fit_transform(flattened_train_data)
    scaled_test_data = scaler.transform(flattened_test_data)

    # Reshape the scaled data back to its original shape
    scaled_train_data = scaled_train_data.reshape(num_samples_train, time_steps_train, num_features)
    scaled_test_data = scaled_test_data.reshape(num_samples_test, time_steps_test, num_features)

    return scaled_train_data, scaled_test_data

def scale_target_var(target_data):
    min_val, max_val = np.min(target_data, axis=0), np.max(target_data, axis=0)
    target_data = (target_data-min_val)/(max_val-min_val)

    return target_data, min_val, max_val

In [26]:
from keras.layers import Input, Dense, Dropout, MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
import sklearn
import numpy as np
import tensorflow as tf

np.set_printoptions(precision=2)
rmse_n=[]
rmse_o=[]
rmse_d=[]
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = Dropout(dropout)(x)
    res = x + inputs

    x = LayerNormalization(epsilon=1e-6)(res)
    x = Dense(ff_dim, activation="relu")(x)
    x = Dropout(dropout)(x)
    x = Dense(inputs.shape[-1])(x)
    return x + res

def transformer_decoder(inputs, enc_outputs, head_size, num_heads, ff_dim, dropout=0):
    # Self attention
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = Dropout(dropout)(x)
    res = x + inputs

    # Cross attention
    x = LayerNormalization(epsilon=1e-6)(res)
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, enc_outputs)
    x = Dropout(dropout)(x)
    res = x + res

    # Feed forward
    x = LayerNormalization(epsilon=1e-6)(res)
    x = Dense(ff_dim, activation="relu")(x)
    x = Dropout(dropout)(x)
    x = Dense(inputs.shape[-1])(x)
    return x + res

def get_transformer(input_shape1, input_shape2):
    enc_inputs = Input(shape=(input_shape1, input_shape2))
    enc_outputs = transformer_encoder(enc_inputs, head_size=64, num_heads=4, ff_dim=128, dropout=0.2)

    dec_inputs = Input(shape=(1, input_shape2))
    dec_outputs = transformer_decoder(dec_inputs, enc_outputs, head_size=64, num_heads=4, ff_dim=128, dropout=0.2)

    x = GlobalAveragePooling1D()(dec_outputs)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(1)(x)
    return Model([enc_inputs, dec_inputs], outputs)

input_groups = [input_group_1, input_group_2, input_group_3, input_group_4, input_group_5]
output_groups = [output_group_1, output_group_2, output_group_3, output_group_4, output_group_5]
ssq_groups = [output_total_ssq_group_1, output_total_ssq_group_2, output_total_ssq_group_3, output_total_ssq_group_4, output_total_ssq_group_5]


# Specify the number of samples to select for each group in each iteration
samples_per_iteration = [
    [3, 3, 3, 3, 2],  # For input_group_1
    [3, 3, 3, 3, 2],  # For input_group_2
    [3, 3, 3, 2, 2],  # For input_group_3
    [2, 2, 2, 2, 3],  # For input_group_4
    [3, 3, 3, 3, 4]   # For input_group_5
]

# Initialize a list of global indices arrays, one for each group
global_indices = [[] for _ in range(len(input_groups))]
print("global_indices",global_indices)

# Outer loop to repeat the sampling process for 5 iterations
for iteration in range(5):
  X_train, X_val, X_test = [], [], []
  y_train, y_val, y_test = [], [], []
  ssq_train, ssq_val, ssq_test = [], [], []
  print(f"Iteration {iteration + 1}")
  print("global_indices",global_indices)
  # Loop over each group
  for i, (input_group, output_group, ssq_group) in enumerate(zip(input_groups, output_groups, ssq_groups)):
      num_samples = samples_per_iteration[i][iteration]  # Number of samples to select for the current group and iteration

      # Create a set of available indices that haven't been selected yet for the current group
      available_indices = list(set(range(len(input_group))) - set(global_indices[i]))

      # Check if there are fewer available indices than needed
      if len(available_indices) < num_samples:
          print(f"Not enough indices left in group {i + 1} to select {num_samples} new samples.")
          num_samples = len(available_indices)  # Adjust to take whatever is left

      # Select the required number of samples from the available indices for the current group
      selected_indices = np.random.choice(available_indices, num_samples, replace=False)
      global_indices[i].extend(selected_indices)  # Add these indices to the group's global list

      # Remove these selected samples from the input, output, and SSQ groups
      X_test_temp = input_group[selected_indices]
      y_test_temp = output_group[selected_indices]
      ssq_test_temp = ssq_group[selected_indices]

      X_temp = np.delete(input_group, selected_indices, axis=0)
      y_temp = np.delete(output_group, selected_indices, axis=0)
      ssq_temp = np.delete(ssq_group, selected_indices, axis=0)

      # Split the remaining data into a training set (60%) and a validation set (40%)
      X_train_temp, X_val_temp, y_train_temp, y_val_temp, ssq_train_temp, ssq_val_temp = train_test_split(
          X_temp, y_temp, ssq_temp, test_size=0.2)

      # Append the results to the corresponding lists
      X_train.append(X_train_temp)
      X_val.append(X_val_temp)
      X_test.append(X_test_temp)

      y_train.append(y_train_temp)
      y_val.append(y_val_temp)
      y_test.append(y_test_temp)

      ssq_train.append(ssq_train_temp)
      ssq_val.append(ssq_val_temp)
      ssq_test.append(ssq_test_temp)

  # After the loop, concatenate the data for all groups if needed
  input_train = np.concatenate(X_train, axis=0)
  input_val = np.concatenate(X_val, axis=0)
  input_test = np.concatenate(X_test, axis=0)

  output_train = np.concatenate(y_train, axis=0)
  output_val = np.concatenate(y_val, axis=0)
  output_test = np.concatenate(y_test, axis=0)

  output_test_total_ssq = np.concatenate(ssq_test, axis=0)


  #  this section for scaling both train and validation set simultaniously
  # Step 1: Combine the training and validation sets
  combined_input = np.concatenate([input_train, input_val], axis=0)
  combined_output = np.concatenate([output_train, output_val], axis=0)

  # Step 2: Scale the combined input data
  # Assuming scale_input_data scales the data based on the combined dataset
  combined_input, input_test = scale_input_data(
      combined_input[:, (60-sample_size):(180-sample_size), :],
      input_test[:, (60-sample_size):(180-sample_size), :]
  )

  # Step 3: Scale the combined output data
  # Assuming scale_target_var scales the data and returns min_val, max_val
  combined_output, min_val, max_val = scale_target_var(combined_output)

  # Step 4: Split the combined data back into training and validation sets
  # Use the original shapes of input_train and input_val to slice the combined arrays
  input_train = combined_input[:input_train.shape[0], :, :]
  input_val = combined_input[input_train.shape[0]:, :, :]

  output_train = combined_output[:output_train.shape[0], :]
  output_val = combined_output[output_train.shape[0]:, :]



  print("input_train :", input_train.shape)
  print("output_train :", output_train.shape)
  print("input_val :", input_val.shape)
  print("output_val :", output_val.shape)
  print("input_test :", input_test.shape)
  print("output_test :", output_test.shape)



  for subclass in range(3):
      # Get input and output layers for Transformer model
      model = get_transformer(120, 47)
      model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))

      # Create decoder inputs (zeros)
      train_dec_input = np.zeros((input_train.shape[0], 1, input_train.shape[2]))
      val_dec_input = np.zeros((input_val.shape[0], 1, input_val.shape[2]))
      test_dec_input = np.zeros((input_test.shape[0], 1, input_test.shape[2]))

      best_val = 1000000
      patience = 0
      best_model = None
      for k in range(200):
          model.fit([input_train, train_dec_input], output_train[:, subclass], epochs=1, batch_size=32)

          pred_val = model.predict([input_val, val_dec_input])
          pred_val = pred_val * (max_val[subclass] - min_val[subclass]) + min_val[subclass]
          loss = sklearn.metrics.mean_squared_error(pred_val, output_val[:, subclass], squared=False)
          if loss <= best_val:
              best_val = loss
              patience = 0
              best_model = model
          else:
              patience += 1
              if patience > 10:
                  print("K:", k)
                  break

      pred_test = best_model.predict([input_test, test_dec_input])
      pred_test = pred_test * (max_val[subclass] - min_val[subclass]) + min_val[subclass]
      loss = sklearn.metrics.mean_squared_error(pred_test, output_test[:,subclass], squared = False)
      if subclass==0:
        rmse_n.append(loss)
      elif subclass==1:
        rmse_o.append(loss)
      else:
        rmse_d.append(loss)
rmse_combined = np.column_stack((rmse_n, rmse_o, rmse_d))
rmse_n_loss = sum(rmse_n) / len(rmse_n)
rmse_o_loss = sum(rmse_o) / len(rmse_o)
rmse_d_loss = sum(rmse_d) / len(rmse_d)
average_rmse = np.array([rmse_n_loss, rmse_o_loss, rmse_d_loss])

print(rmse_n_loss,rmse_o_loss,rmse_d_loss)

global_indices [[], [], [], [], []]
Iteration 1
global_indices [[], [], [], [], []]
input_train : (41, 120, 47)
output_train : (41, 3)
input_val : (13, 120, 47)
output_val : (13, 3)
input_test : (14, 120, 47)
output_test : (14, 3)


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (None, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 60ms/step - loss: 0.2873


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (13, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.1612
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - loss: 0.1651
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - loss: 0.0857
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - loss: 0.1400
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - loss: 0.0668
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - loss: 0.1193
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - loss: 0.0820
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.0731
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.0637
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0413
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss:

...

2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 66ms/step - loss: 0.1530


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (14, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 410ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.1651
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.1391
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0920
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.1207
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.0535
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.0895
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.0849
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.0859
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.0572
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - loss: 0.0730
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - loss: 0.0

/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (None, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 440ms/step
3.857845403163407 2.403508474844188 2.9044800806775486
